In [1]:
import pandas as pd
import numpy as np
import warnings
from math import sqrt
warnings.filterwarnings('ignore')
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.train.automl import AutoMLConfig
import pickle
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import mlflow

from azureml.core import Workspace, Dataset

In [3]:
subscription_id = '292890d4-aa6d-4d5e-a085-97c80db3c30a'
resource_group = 'MLOpsRG'
workspace_name = 'MLOpsWS'

workspace = Workspace(subscription_id, resource_group, workspace_name)
print(workspace)

Workspace.create(name='MLOpsWS', subscription_id='292890d4-aa6d-4d5e-a085-97c80db3c30a', resource_group='MLOpsRG')


In [4]:
uri = workspace.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(uri)
print(uri)

azureml://australiaeast.api.azureml.ms/mlflow/v1.0/subscriptions/292890d4-aa6d-4d5e-a085-97c80db3c30a/resourceGroups/MLOpsRG/providers/Microsoft.MachineLearningServices/workspaces/MLOpsWS?


# Train XGBRegressor using splitData1

In [ ]:
df = pd.read_csv("Dataset/splitData1_kfold_TrainAndValid.csv")
df.head()

In [ ]:
df.shape

# Using KFolds for cross validation

In [ ]:
xgb_params = {
              'random_state': 1,
              'n_estimators': 20000,
              'objective': 'reg:squarederror',
              'n_jobs': -1,                 
              # Only test with these parameters
              'gpu_id': 0,
              'predictor': 'gpu_predictor',
              'tree_method': 'gpu_hist',
              # Only test with these parameters
              }

In [ ]:
folds=5
total_mean_rmse = 0
total_mean_mae = 0

for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.SalePrice
    yvalid = xvalid.SalePrice

    model = XGBRegressor(**xgb_params)
    
    # fit, validate and score       
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
    # If True returns MSE value, if False returns RMSE value.
    fold_rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    fold_mae = mean_absolute_error(yvalid, preds_valid)

    print(fold, fold_rmse)
    print(fold, fold_mae)
   
    # calculate mean RMSE and MAE
    total_mean_rmse += fold_rmse / folds
    total_mean_mae += fold_mae / folds
  
print(f"Total Mean RMSE: {total_mean_rmse}")
print(f"Total Mean MAE: {total_mean_mae}")

In [ ]:
# Total Mean RMSE: 428.03259475713577
# Total Mean MAE: 82.72462039356864